# Technical Factor - Bollinger Bands of Bull and Bear Index (BBIBOLL)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        close,
        (m_avg(close, 3) + m_avg(close, 6) + m_avg(close, 12) + m_avg(close, 24)) / 4 AS BBIBOLL_MIDDLE,
        BBIBOLL_MIDDLE + 2 * m_stddev(BBIBOLL_MIDDLE, 10) AS BBIBOLL_UPPER,
        BBIBOLL_MIDDLE - 2 * m_stddev(BBIBOLL_MIDDLE, 10) AS BBIBOLL_LOWER,
        (close - BBIBOLL_MIDDLE) / m_stddev(BBIBOLL_MIDDLE, 10) AS Z,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(close > BBIBOLL_MIDDLE AND m_lag(close, 1) <= m_lag(BBIBOLL_MIDDLE, 1), 1, 0) AS TRBY1,
        IF(close < BBIBOLL_MIDDLE AND m_lag(close, 1) >= m_lag(BBIBOLL_MIDDLE, 1), 1, 0) AS TRSL1,
        IF(BBIBOLL_MIDDLE > m_lag(BBIBOLL_MIDDLE, 1), 1, 0) AS TRBY2,
        IF(BBIBOLL_MIDDLE < m_lag(BBIBOLL_MIDDLE, 1), 1, 0) AS TRSL2,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(close > BBIBOLL_MIDDLE AND (close - BBIBOLL_MIDDLE) > (m_lag(close, 1) - m_lag(BBIBOLL_MIDDLE, 1)), 1, 0) AS MTBY1,
        IF(close < BBIBOLL_MIDDLE AND (close - BBIBOLL_MIDDLE) < (m_lag(close, 1) - m_lag(BBIBOLL_MIDDLE, 1)), 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(close < BBIBOLL_LOWER AND close > m_lag(close, 1), 1, 0) AS RVBY1,
        IF(close > BBIBOLL_UPPER AND close < m_lag(close, 1), 1, 0) AS RVSL1,
        IF(Z < -2 AND (Z - m_lag(Z, 1)) > 0, 1, 0) AS RVBY2,
        IF(Z > 2 AND (Z - m_lag(Z, 1)) < 0, 1, 0) AS RVSL2,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(close, 1) <= m_lag(BBIBOLL_UPPER, 1) AND close > BBIBOLL_UPPER, 1, 0) AS BKBY1,
        IF(m_lag(close, 1) >= m_lag(BBIBOLL_LOWER, 1) AND close < BBIBOLL_LOWER, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        close,
        BBIBOLL_MIDDLE,
        BBIBOLL_UPPER,
        BBIBOLL_LOWER,
        Z,
        TRBY1,
        TRSL1,
        TRBY2,
        TRSL2,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        RVBY2,
        RVSL2,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend    USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,close,BBIBOLL_MIDDLE,BBIBOLL_UPPER,BBIBOLL_LOWER,Z,TRBY1,TRSL1,TRBY2,TRSL2,MTBY1,MTSL1,RVBY1,RVSL1,RVBY2,RVSL2,BKBY1,BKSL1
0,2025-02-25,600519.SH,11844.153973,11950.284157,12152.581967,11747.986346,-1.049247,0,1,0,1,0,1,0,0,0,0,0,0
1,2025-02-26,600519.SH,11893.110896,11929.934654,12094.621763,11765.247545,-0.447197,0,0,0,1,0,0,0,0,0,0,0,0
2,2025-02-27,600519.SH,12101.238911,11947.283746,12072.318986,11822.248507,2.462588,1,0,1,0,1,0,0,0,0,0,1,0
3,2025-02-28,600519.SH,12225.301129,12001.852862,12108.436770,11895.268954,4.192908,0,0,1,0,1,0,0,0,0,0,0,0
4,2025-03-03,600519.SH,12113.131940,12024.547708,12120.650089,11928.445328,1.843539,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,2025-12-25,600519.SH,11945.482630,11877.204426,11914.222408,11840.186444,3.688921,1,0,1,0,1,0,0,0,0,0,1,0
207,2025-12-26,600519.SH,11945.144750,11889.047573,11924.305856,11853.789290,3.182071,0,0,1,0,0,0,0,1,0,1,0,0
208,2025-12-29,600519.SH,11842.682738,11887.784217,11920.086892,11855.481541,-2.792430,0,1,0,1,0,1,0,0,0,0,0,1
209,2025-12-30,600519.SH,11738.953676,11859.696851,11884.045788,11835.347915,-9.917737,0,0,0,1,0,1,0,0,0,0,0,0
